In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/run1.csv')  # Replace 'your_file.csv' with the actual file name

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/run2.csv')  # Replace 'your_file.csv' with the actual file name

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/run3.csv')  # Replace 'your_file.csv' with the actual file name

In [ ]:
df4 = pd.read_csv('/content/drive/MyDrive/run4.csv')  # Replace 'your_file.csv' with the actual file name

In [ ]:
df1.to_csv('YenCs_task1_run1.tsv', sep='\t', index=False)
df2.to_csv('YenCs_task1_run2.tsv', sep='\t', index=False)
df3.to_csv('YenCs_task1_run3.tsv', sep='\t', index=False)
df4.to_csv('YenCs_task1_run4.tsv', sep='\t', index=False)
  # Replace 'your_file.tsv' with the desired file name

In [ ]:
# Assuming your DataFrame is named 'submission_df'
submission_df1.to_csv('YenCs_task1_run1.tsv', sep='\t', index=False)
submission_df2.to_csv('YenCs_task1_run2.tsv', sep='\t', index=False)
submission_df3.to_csv('YenCs_task1_run3.tsv', sep='\t', index=False)
submission_df4.to_csv('YenCs_task1_run4.tsv', sep='\t', index=False)

NameError: name 'submission_df1' is not defined

Import libraries and packages




In [17]:
!pip install tensorflow
from tensorflow.keras.layers import Input, Embedding, Bidirectional, SimpleRNN, Dense
from tensorflow.keras.models import Model

In [19]:
!pip install tensorflow
from tensorflow.keras.layers import Input, Embedding, Bidirectional, SimpleRNN, Dense # Import Input here
from tensorflow.keras.models import Model

In [75]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping
from textblob import TextBlob

import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences # Import pad_sequences

**Data** **loading**

In [3]:
data_train=pd.read_csv("/content/drive/MyDrive/Fake_train.csv")
data_dev=pd.read_csv("/content/drive/MyDrive/Fake_dev.csv")
data_train.head()

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original


## ***preprocessing***

In [2]:
def clean_text(text):
    stop_words=open(r'/content/drive/MyDrive/icon_task_2020/malayalam_stopwords.txt', encoding='utf=8').read().split('\n')
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))
    text = "".join([word for word in text if word not in string.punctuation])
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\s+',text)
    text = " ".join(word for word in tokens if word not in stop_words)          # here english words may lemmatize so kept lematization
    return text

In [8]:
data_train['clean'] = data_train['text'].apply(lambda x:clean_text(x))
data_dev['clean'] = data_dev['text'].apply(lambda x:clean_text(x))
data_test['clean'] = data_test['text'].apply(lambda x:clean_text(x))

In [9]:
data_train.value_counts('label')

,count
label,
original,1658
Fake,1599


In [10]:
data_dev.value_counts('label')

,count
label,
original,409
Fake,406


In [12]:
X_train=data_train['clean']
X_dev=data_dev['clean']
X_test= data_test['clean']

X_train=X_train.tolist()
X_dev=X_dev.tolist()
X_test=X_test.tolist()

In [13]:


vocabulary_size = 30000  #
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train1 = pad_sequences(sequences, maxlen=100)
sequences = tokenizer.texts_to_sequences(X_dev)
X_test1 = pad_sequences(sequences, maxlen=100)

sequences = tokenizer.texts_to_sequences(X_test)
X_test_witout= pad_sequences(sequences, maxlen=100)

**Label Encoding**

In [14]:
y_train_sent=data_train['label']
#y_train=data_train['label']
y_test_sent=data_dev['label']
y = pd.get_dummies(data_train['label']).values


y_train1=pd.get_dummies(y_train_sent).values                                #get_dummies() is used for data manipulation. It converts categorical data into dummy or indicator variables.
y_test1=pd.get_dummies(y_test_sent).values
diz_label1 = {}
for i,label in enumerate(data_train.label.factorize()[1]):             #enumerate() allows us to iterate through a sequence but it keeps track of both the index and the element
    diz_label1[i] = label

diz_label1





{0: 'Fake', 1: 'original'}

**FastText Embedding**

In [15]:
#Convertin
embeddings_index = {}
with open("/content/drive/MyDrive/icon_task_2020/wiki.ml.vec",encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

<ipython-input-15-cdc48852e23e>:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


Found 189665 word vectors.


In [16]:
VOCAB_SIZE = len(tokenizer.word_index) + 2
EMBEDDING_DIM = 300
hits = 0
misses =0

# Prepare embedding matrix
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        misses += 1

print("Converted %d words (%d misses)" % (hits, misses))

Converted 7048 words (10158 misses)


Bi-RNN Model

In [20]:
def create_birnn_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM):
    #Input Layer is defined within tensorflow.keras.layers. We import it above.
    input_layer = Input(shape=(X_train1.shape[1],), name="Input_layer")
    embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix],
                                trainable=True, input_length=X_train1.shape[1],
                                name="Embedding_layer")(input_layer)
    rnn_layer = Bidirectional(SimpleRNN(128))(embedding_layer)
    dense_layer = Dense(128, activation='relu')(rnn_layer)
    output_layer = Dense(2, activation='sigmoid')(dense_layer)  # Modified output layer
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Optional: Change loss function
    return model

In [21]:
birnn_model = create_birnn_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
 #Import EarlyStopping from tensorflow.keras.callbacks
birnn_model_history = birnn_model.fit(X_train1, y_train1, epochs=40, batch_size=225,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.6037 - loss: 0.6640 - val_accuracy: 0.6380 - val_loss: 0.6674
Epoch 2/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 911ms/step - accuracy: 0.7552 - loss: 0.5341 - val_accuracy: 0.6687 - val_loss: 0.5989
Epoch 3/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 931ms/step - accuracy: 0.7632 - loss: 0.5016 - val_accuracy: 0.5828 - val_loss: 0.7052
Epoch 4/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 877ms/step - accuracy: 0.7472 - loss: 0.5178 - val_accuracy: 0.5890 - val_loss: 0.6980
Epoch 5/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 659ms/step - accuracy: 0.8520 - loss: 0.3517 - val_accuracy: 0.6166 - val_loss: 0.7352


In [24]:
pred_birnn = birnn_model.predict(X_test1)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [25]:
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(pred_birnn,axis=1)))

              precision    recall  f1-score   support

           0       0.71      0.64      0.67       406
           1       0.67      0.74      0.71       409

    accuracy                           0.69       815
   macro avg       0.69      0.69      0.69       815
weighted avg       0.69      0.69      0.69       815



DNN

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

def create_dnn_model(X_train, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM):
    input_layer = Input(shape=(X_train.shape[1],), name="Input_layer")
    embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix],
                                trainable=True, input_length=X_train.shape[1],
                                name="Embedding_layer")(input_layer)
    flatten_layer = Flatten()(embedding_layer)  # Flatten the embedding output
    dense_layer1 = Dense(256, activation='relu')(flatten_layer)  # First hidden layer
    dropout_layer = Dropout(0.5)(dense_layer1)  # Dropout for regularization
    dense_layer2 = Dense(128, activation='relu')(dropout_layer)  # Second hidden layer
    output_layer = Dense(2, activation='sigmoid')(dense_layer2)  # Output layer with sigmoid for binary classification
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Compile with appropriate loss and metrics
    return model

In [27]:
dnn_model = create_dnn_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
dnn_model_history = dnn_model.fit(X_train1, y_train1, epochs=100, batch_size=225,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 792ms/step - accuracy: 0.5631 - loss: 0.6782 - val_accuracy: 0.6564 - val_loss: 0.6318
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 636ms/step - accuracy: 0.7466 - loss: 0.5600 - val_accuracy: 0.6810 - val_loss: 0.6304
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 730ms/step - accuracy: 0.8188 - loss: 0.4045 - val_accuracy: 0.6871 - val_loss: 0.5897
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 689ms/step - accuracy: 0.9148 - loss: 0.2258 - val_accuracy: 0.7086 - val_loss: 0.6436
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 674ms/step - accuracy: 0.9678 - loss: 0.1166 - val_accuracy: 0.6779 - val_loss: 0.7683
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 788ms/step - accuracy: 0.9873 - loss: 0.0495 - val_accuracy: 0.6933 - val_loss: 0.9758


In [29]:
#predictions on Dev
pred_dnn = dnn_model.predict(X_test1) # Predict using the created model

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step


In [30]:
# Prediction on Test
pred_dnn = dnn_model.predict(X_test_witout) # Predict using the created model

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [ ]:
# pred_lr=model_lr.predict(X_test_witout)


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step


In [31]:
from tensorflow.keras.models import Sequential # Import the Sequential class from Keras
from tensorflow.keras.layers import LSTM # Import the LSTM layer

model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model.add(LSTM(128, return_sequences=True))  # LSTM layer with return_sequences=True
model.add(SimpleRNN(64))  # RNN layer after LSTM
model.add(Dense(2, activation='sigmoid'))  # Output layer for binary classification

In [39]:
def GRU_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM):
    Input_layer = Input(shape=(X_train1.shape[1],), name="Input_layer")
    Embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True,
                                input_length=X_train1.shape[1], name="Embedding_layer")(Input_layer)
    GRU_layer1 = GRU(128, return_sequences=False, dropout=0.2)(Embedding_layer) # Changed return_sequences to False
    Dense_layer1 = Dense(128, activation='relu', name="Dense_layer1a")(GRU_layer1)
    predictions_task1 = Dense(2, activation='sigmoid', name="predictions_task1")(Dense_layer1)  # Binary classification output layer
    model = tf.keras.models.Model(Input_layer, predictions_task1)

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])  # Compile with binary_crossentropy
    return model

In [40]:
def GRU_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM):
    Input_layer = Input(shape=(X_train1.shape[1],), name="Input_layer")
    Embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True,
                                input_length=X_train1.shape[1], name="Embedding_layer")(Input_layer)
    GRU_layer1 = GRU(128, return_sequences=False, dropout=0.2)(Embedding_layer)
    Dense_layer1 = Dense(128, activation='relu', name="Dense_layer1a")(GRU_layer1)
    predictions_task1 = Dense(2, activation='sigmoid', name="predictions_task1")(Dense_layer1)
    model = tf.keras.models.Model(Input_layer, predictions_task1)

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])
    return model


In [42]:
gru_model = GRU_model(X_train1, embedding_matrix, VOCAB_SIZE, EMBEDDING_DIM)
gru_model_history = gru_model.fit(X_train1, y_train1, epochs=40, batch_size=225,
                                 validation_split=0.1,
                                 callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

pred_gru = gru_model.predict(X_test1)

print(classification_report(np.argmax(y_test1, axis=1), np.argmax(pred_gru, axis=1)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.5716 - loss: 0.6716 - val_accuracy: 0.6043 - val_loss: 0.6585
Epoch 2/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6681 - loss: 0.6063 - val_accuracy: 0.6503 - val_loss: 0.6898
Epoch 3/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.7238 - loss: 0.5586 - val_accuracy: 0.6595 - val_loss: 0.6583
Epoch 4/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.7506 - loss: 0.5164 - val_accuracy: 0.6779 - val_loss: 0.6028
Epoch 5/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.7829 - loss: 0.4709 - val_accuracy: 0.7178 - val_loss: 0.6735
Epoch 6/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.8019 - loss: 0.4366 - val_accuracy: 0.7117 - val_loss: 0.7652
Epoch 7/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 948ms/step - accuracy: 0.8024 - loss: 0.4482 - val_accuracy: 0.7270 - val_loss: 0.5366
Epoch 8/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 878ms/step - accuracy: 0.8322 - loss: 0.3870 - val_accuracy: 0.6963 - val

In [43]:
# pred_gru = gru_model.predict(X_test_witout)


**Fusion Model**

In [77]:
# val_accuracy_birnn = birnn_model_history.history['val_accuracy'][-1] # Get the last value of validation accuracy
# val_accuracy_dnn = dnn_model_history.history['val_accuracy'][-1]
# val_accuracy_gru = gru_model_history.history['val_accuracy'][-1]

# total_accuracy = val_accuracy_birnn + val_accuracy_dnn + val_accuracy_gru + val_accuracy_lstm_rnn
# weight_birnn = val_accuracy_birnn / total_accuracy
# weight_dnn = val_accuracy_dnn / total_accuracy
# weight_gru = val_accuracy_gru / total_accuracy

# weight_lstm_rnn = val_accuracy_lstm_rnn / total_accuracy  # Weight for LSTM + RNN
# print(f"BiRNN Weight: {weight_birnn}")
# print(f"DNN Weight: {weight_dnn}")
# print(f"GRU Weight: {weight_gru}")
# print(f"LSTM + RNN Weight: {weight_lstm_rnn}")

In [76]:
# val_accuracy_birnn = birnn_model_history.history['val_accuracy'][-1] # Get the last value of validation accuracy
# val_accuracy_dnn = dnn_model_history.history['val_accuracy'][-1]
# val_accuracy_gru = gru_model_history.history['val_accuracy'][-1]

# total_accuracy = val_accuracy_birnn + val_accuracy_dnn + val_accuracy_gru + val_accuracy_lstm_rnn
# weight_birnn = val_accuracy_birnn / total_accuracy
# weight_dnn = val_accuracy_dnn / total_accuracy
# weight_gru = val_accuracy_gru / total_accuracy

# weight_lstm_rnn = val_accuracy_lstm_rnn / total_accuracy  # Weight for LSTM + RNN
# print(f"BiRNN Weight: {weight_birnn}")
# print(f"DNN Weight: {weight_dnn}")
# print(f"GRU Weight: {weight_gru}")
# print(f"LSTM + RNN Weight: {weight_lstm_rnn}")

In [53]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Compile the model
history = model.fit(X_train1, y_train1, epochs=40, batch_size=225, validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

val_accuracy_lstm_rnn = history.history['val_accuracy'][-1]

# Rest of your code for calculating weights and ensemble predictions
total_accuracy = val_accuracy_birnn + val_accuracy_dnn + val_accuracy_gru + val_accuracy_lstm_rnn
weight_birnn = val_accuracy_birnn / total_accuracy
weight_dnn = val_accuracy_dnn / total_accuracy
weight_gru = val_accuracy_gru / total_accuracy

weight_lstm_rnn = val_accuracy_lstm_rnn / total_accuracy  # Weight for LSTM + RNN
print(f"BiRNN Weight: {weight_birnn}")
print(f"DNN Weight: {weight_dnn}")
print(f"GRU Weight: {weight_gru}")
print(f"LSTM + RNN Weight: {weight_lstm_rnn}")

Epoch 1/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6054 - loss: 0.6546 - val_accuracy: 0.6718 - val_loss: 0.5958
Epoch 2/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.7703 - loss: 0.5116 - val_accuracy: 0.6933 - val_loss: 0.5478
Epoch 3/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 947ms/step - accuracy: 0.8506 - loss: 0.3574 - val_accuracy: 0.7454 - val_loss: 0.4868
Epoch 4/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8863 - loss: 0.2672 - val_accuracy: 0.7270 - val_loss: 0.5112
Epoch 5/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 988ms/step - accuracy: 0.9423 - loss: 0.1660 - val_accuracy: 0.7607 - val_loss: 0.5373
Epoch 6/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9670 - loss: 0.1206 - val_accuracy: 0.7945 - val_loss: 0.5150
BiRNN Weight: 0.21776813927144817
DNN Weight: 0.24485374412940356
GRU Weight: 0.2567713959514568
LSTM + RNN Weight: 0.28060672064769143


In [ ]:
# ipython-input-116-7ddd3cfe2736
predicted_class_indices = decoded_predictions_en  # decoded_predictions_en already contains the predicted class indices
decoded_labels_en = [diz_label1[index] for index in predicted_class_indices]

In [ ]:
len(submission_df)

1019

In [ ]:
len(submission_df)

1019

In [ ]:
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(pred_dnn,axis=1)))

              precision    recall  f1-score   support

           0       0.84      0.69      0.76       406
           1       0.74      0.87      0.80       409

    accuracy                           0.78       815
   macro avg       0.79      0.78      0.78       815
weighted avg       0.79      0.78      0.78       815



In [ ]:
len(submission_df)

1019

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.models import Model

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step


              precision    recall  f1-score   support

           0       0.94      0.42      0.58       406
           1       0.63      0.97      0.76       409

    accuracy                           0.70       815
   macro avg       0.78      0.70      0.67       815
weighted avg       0.78      0.70      0.67       815



In [ ]:
val_accuracy_lstm_rnn = history.history['val_accuracy'][-1]
print(f"LSTM + RNN Validation Accuracy: {val_accuracy_lstm_rnn}")

In [ ]:
pred_lstm_rnn = model.predict(X_test1) # Get predictions from LSTM + RNN
ensemble_predictions = weight_birnn * pred_birnn + weight_dnn * pred_dnn + weight_gru * pred_gru + weight_lstm_rnn * pred_lstm_rnn
final_predictions = np.argmax(ensemble_predictions, axis=1)

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step


In [ ]:
# ... (Code for training birnn_model, dnn_model, and gru_model) ...

# Extract validation accuracies from model histories
val_accuracy_birnn = birnn_model_history.history['val_accuracy'][-1]
val_accuracy_dnn = dnn_model_history.history['val_accuracy'][-1]
val_accuracy_gru = gru_model_history.history['val_accuracy'][-1]

# ... (Rest of your ensemble code) ...

total_accuracy = val_accuracy_birnn + val_accuracy_dnn + val_accuracy_gru + val_accuracy_lstm_rnn

# ... (Calculate weights and ensemble predictions) ...

In [ ]:
print(classification_report(np.argmax(y_test1,axis=1), final_predictions))

              precision    recall  f1-score   support

           0       0.89      0.67      0.76       406
           1       0.73      0.91      0.81       409

    accuracy                           0.79       815
   macro avg       0.81      0.79      0.79       815
weighted avg       0.81      0.79      0.79       815



In [54]:
from tensorflow.keras.models import Sequential # Import the Sequential class from Keras
from tensorflow.keras.layers import LSTM # Import the LSTM layer

model_lr = Sequential()
model_lr.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model_lr.add(LSTM(128, return_sequences=True))  # LSTM layer with return_sequences=True
model_lr.add(SimpleRNN(64))  # RNN layer after LSTM
model_lr.add(Dense(2, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model_lr.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model and store the training history in 'history'
history = model_lr.fit(X_train1, y_train1, epochs=40, batch_size=225, validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

val_accuracy_lstm_rnn = history.history['val_accuracy'][-1]
print(f"LSTM + RNN Validation Accuracy: {val_accuracy_lstm_rnn}")

Epoch 1/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5982 - loss: 0.6518 - val_accuracy: 0.6810 - val_loss: 0.6050
Epoch 2/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7636 - loss: 0.5043 - val_accuracy: 0.7117 - val_loss: 0.5758
Epoch 3/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8170 - loss: 0.4156 - val_accuracy: 0.7270 - val_loss: 0.5169
Epoch 4/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8920 - loss: 0.2696 - val_accuracy: 0.7607 - val_loss: 0.5507
Epoch 5/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9242 - loss: 0.2074 - val_accuracy: 0.7699 - val_loss: 0.5133
Epoch 6/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9657 - loss: 0.1217 - val_accuracy: 0.6840 - val_loss: 0.6322
Epoch 7/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9528 - loss: 0.1219 - val_accuracy: 0.7669 - val_loss: 0.5093
Epoch 8/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9904 - loss: 0.0627 - val_accuracy: 0.7853 - val_loss:

In [55]:
model_lr.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
history = model_lr.fit(X_train1, y_train1, epochs=40, batch_size=225,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9951 - loss: 0.0209 - val_accuracy: 0.7761 - val_loss: 0.7570
Epoch 2/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 967ms/step - accuracy: 0.9961 - loss: 0.0131 - val_accuracy: 0.7945 - val_loss: 0.8122
Epoch 3/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9992 - loss: 0.0052 - val_accuracy: 0.7699 - val_loss: 0.9691
Epoch 4/40
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9963 - loss: 0.0097 - val_accuracy: 0.7730 - val_loss: 0.9431


In [57]:
pred_lr=model_lr.predict(X_test1)


26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step


In [58]:
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(pred_lr,axis=1)))

              precision    recall  f1-score   support

           0       0.85      0.76      0.80       406
           1       0.78      0.87      0.83       409

    accuracy                           0.81       815
   macro avg       0.82      0.81      0.81       815
weighted avg       0.82      0.81      0.81       815



In [63]:
val_accuracy_birnn = birnn_model_history.history['val_accuracy'][-1] # Get the last value of validation accuracy
val_accuracy_dnn = dnn_model_history.history['val_accuracy'][-1]
val_accuracy_gru = gru_model_history.history['val_accuracy'][-1]

print(f"BiRNN Validation Accuracy: {val_accuracy_birnn}")
print(f"DNN Validation Accuracy: {val_accuracy_dnn}")
print(f"GRU Validation Accuracy: {val_accuracy_gru}")

BiRNN Validation Accuracy: 0.6165643930435181
DNN Validation Accuracy: 0.6932515501976013
GRU Validation Accuracy: 0.7269938588142395


In [60]:
total_accuracy = val_accuracy_birnn + val_accuracy_dnn + val_accuracy_gru

weight_birnn = val_accuracy_birnn / total_accuracy
weight_dnn = val_accuracy_dnn / total_accuracy
weight_gru = val_accuracy_gru / total_accuracy

print(f"BiRNN Weight: {weight_birnn}")
print(f"DNN Weight: {weight_dnn}")
print(f"GRU Weight: {weight_gru}")

BiRNN Weight: 0.30271083358953726
DNN Weight: 0.3403614561840955
GRU Weight: 0.3569277102263672


In [61]:
pred_birnn = birnn_model.predict(X_test1)
pred_dnn = dnn_model.predict(X_test1)
pred_gru = gru_model.predict(X_test1)

ensemble_predictions = weight_birnn * pred_birnn + weight_dnn * pred_dnn + weight_gru * pred_gru

# You might need to adjust this based on how your models output predictions
final_predictions = np.argmax(ensemble_predictions, axis=1)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step


In [62]:
print(classification_report(np.argmax(y_test1,axis=1), final_predictions))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76       406
           1       0.74      0.88      0.80       409

    accuracy                           0.79       815
   macro avg       0.80      0.78      0.78       815
weighted avg       0.80      0.79      0.78       815



In [64]:
ensemble_predictions_birnn_dnn = weight_birnn * pred_birnn + weight_dnn * pred_dnn
final_predictions_birnn_dnn = np.argmax(ensemble_predictions_birnn_dnn, axis=1)
print(classification_report(np.argmax(y_test1,axis=1), final_predictions_birnn_dnn))

              precision    recall  f1-score   support

           0       0.73      0.83      0.78       406
           1       0.81      0.69      0.74       409

    accuracy                           0.76       815
   macro avg       0.77      0.76      0.76       815
weighted avg       0.77      0.76      0.76       815



In [65]:
ensemble_predictions_birnn_gru = weight_birnn * pred_birnn + weight_gru * pred_gru
final_predictions_birnn_gru = np.argmax(ensemble_predictions_birnn_gru, axis=1)
print(classification_report(np.argmax(y_test1,axis=1), final_predictions_birnn_gru))

              precision    recall  f1-score   support

           0       0.91      0.59      0.72       406
           1       0.70      0.94      0.80       409

    accuracy                           0.77       815
   macro avg       0.80      0.77      0.76       815
weighted avg       0.80      0.77      0.76       815



In [66]:
ensemble_predictions_dnn_gru = weight_dnn * pred_dnn + weight_gru * pred_gru
final_predictions_dnn_gru = np.argmax(ensemble_predictions_dnn_gru, axis=1)
print(classification_report(np.argmax(y_test1,axis=1), final_predictions_dnn_gru))

              precision    recall  f1-score   support

           0       0.84      0.72      0.78       406
           1       0.76      0.87      0.81       409

    accuracy                           0.80       815
   macro avg       0.80      0.80      0.80       815
weighted avg       0.80      0.80      0.80       815



In [67]:
pred_dnn = dnn_model.predict(X_test_witout)  # Generate predictions for the same data (X_test1)
pred_lstm_rnn = model.predict(X_test_witout) # Generate predictions for the same data (X_test1)

ensemble_predictions_dnn_lstm_rnn = weight_dnn * pred_dnn + weight_lstm_rnn * pred_lstm_rnn
final_predictions_dnn_lstm_rnn = np.argmax(ensemble_predictions_dnn_lstm_rnn, axis=1)
# print(classification_report(np.argmax(y_test1,axis=1), final_predictions_dnn_lstm_rnn))

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step


In [68]:
# Assuming 'X_test1' is the correct data for both models, use it for prediction:
pred_birnn = birnn_model.predict(X_test_witout)
pred_lstm_rnn = model.predict(X_test_witout)

# Now, perform the ensemble prediction:
ensemble_predictions_birnn_lstm_rnn = weight_birnn * pred_birnn + weight_lstm_rnn * pred_lstm_rnn
final_predictions_birnn_lstm_rnn = np.argmax(ensemble_predictions_birnn_lstm_rnn, axis=1)
# print(classification_report(np.argmax(y_test1,axis=1), final_predictions_birnn_lstm_rnn))

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 151ms/step


In [69]:
ensemble_predictions_dnn_lstm_rnn = weight_dnn * pred_dnn + weight_lstm_rnn * pred_lstm_rnn

In [72]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model.add(GRU(128, return_sequences=True))  # GRU layer with return_sequences=True to pass output to RNN
model.add(SimpleRNN(64))  # RNN layer after GRU
model.add(Dense(2, activation='sigmoid'))  # Output layer for binary classification (adjust units as needed)

In [73]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
history = model.fit(X_train1, y_train1, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 33s 804ms/step - accuracy: 0.9821 - loss: 0.0660 - val_accuracy: 0.8742 - val_loss: 0.3550
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 38s 928ms/step - accuracy: 0.9923 - loss: 0.0326 - val_accuracy: 0.8834 - val_loss: 0.4121
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 30s 648ms/step - accuracy: 0.9960 - loss: 0.0216 - val_accuracy: 0.8819 - val_loss: 0.3942
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 33s 456ms/step - accuracy: 0.9985 - loss: 0.0084 - val_accuracy: 0.9049 - val_loss: 0.3734
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 27s 627ms/step - accuracy: 0.9974 - loss: 0.0082 - val_accuracy: 0.9049 - val_loss: 0.4041
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 37s 537ms/step - accuracy: 0.9688 - loss: 0.1070 - val_accuracy: 0.7669 - val_loss: 0.5314
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 39s 483ms/step - accuracy: 0.9619 - loss: 0.1275 - val_accuracy: 0.8727 - val_loss: 0.3375
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 448ms/step - accuracy: 0.9958 - loss: 0.0173 - val_accu

In [ ]:
pred = model.predict(X_test_witout)  # Get the predictions
# Assuming your model outputs probabilities, get the class with highest probability
# Replace 'axis=1' if your model outputs a different shape
predicted_classes = np.argmax(pred, axis=1)

# Evaluate using appropriate metrics (e.g., classification_report)
# print(classification_report(np.argmax(y_test1, axis=1), predicted_classes))

32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step


Run submission

In [ ]:
# Get predictions for the development set using both models
pred_dnn_dev = dnn_model.predict(X_test1)  # Predictions for development set using dnn_model
pred_lstm_rnn_dev = model.predict(X_test1) # Predictions for development set using model (LSTM + RNN)

# Now perform the ensemble on the development set predictions
ensemble_predictions_dnn_lstm_rnn = weight_dnn * pred_dnn_dev + weight_lstm_rnn * pred_lstm_rnn_dev
final_predictions_dnn_lstm_rnn = np.argmax(ensemble_predictions_dnn_lstm_rnn, axis=1)

# ... (rest of your code for evaluation on the development set) ...


# For submission, get predictions for the test set without labels
pred_dnn_test = dnn_model.predict(X_test_witout)  # Predictions for test set using dnn_model
pred_lstm_rnn_test = model.predict(X_test_witout) # Predictions for test set using model (LSTM + RNN)

# Perform the ensemble for the test set predictions
ensemble_predictions_test = weight_dnn * pred_dnn_test + weight_lstm_rnn * pred_lstm_rnn_test
final_predictions_test = np.argmax(ensemble_predictions_test, axis=1)

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'id': data_test['Id'],
    'label': final_predictions_test  # Use the test set predictions for submission
})

submission_df.to_csv('run2.csv', index=False)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step
